In [1]:
import gensim
import pandas as pd
import nltk
import numpy as np
import re
from scipy.spatial import distance
from nltk.corpus import stopwords

In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
stop_words = stopwords.words('english')

In [3]:
model.vectors.shape

(3000000, 300)

In [4]:
def preprocess(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower().split()
    stopwords_set = set(stopwords.words("english"))
    text = list(set([w for w in text if w not in stopwords_set]))
    
    s = []
    for i in text:
        if i in model.vocab:
            s.append(i);
    
    return s

In [5]:
M1 = "Introduction of Deadlock in Operating System A process in operating systems uses different resources and uses resources in following way. 1) Requests a resource 2) Use the resource 2) Releases the resource  Deadlock is a situation where a set of processes are blocked because each process is holding a resource and waiting for another resource acquired by some other process. Consider an example when two trains are coming toward each other on same track and there is only one track, none of the trains can move once they are in front of each other. Similar situation occurs in operating systems when there are two or more processes hold some resources and wait for resources held by other(s). For example, in the below diagram, Process 1 is holding Resource 1 and waiting for resource 2 which is acquired by process 2, and process 2 is waiting for resource 1.  Deadlock can arise if following four conditions hold simultaneously (Necessary Conditions) Mutual Exclusion: One or more than one resource are non-sharable (Only one process can use at a time) Hold and Wait: A process is holding at least one resource and waiting for resources. No Preemption: A resource cannot be taken from a process unless the process releases the resource. Circular Wait: A set of processes are waiting for each other in circular form. Methods for handling deadlock There are three ways to handle deadlock 1) Deadlock prevention or avoidance: The idea is to not let the system into deadlock state. One can zoom into each category individually, Prevention is done by negating one of above mentioned necessary conditions for deadlock. Avoidance is kind of futuristic in nature. By using strategy of “Avoidance”, we have to make an assumption. We need to ensure that all information about resources which process WILL need are known to us prior to execution of the process. We use Banker’s algorithm (Which is in-turn a gift from) in order to avoid deadlock.  2) Deadlock detection and recovery: Let deadlock occur, then do preemption to handle it once occurred.  3) Ignore the problem all together: If deadlock is very rare, then let it happen and reboot the system. This is the approach that both Windows and UNIX take."
M2 = "A thread is the smallest unit of processing that can be performed in an OS. In most modern operating systems, a thread exists within a process - that is, a single process may contain multiple threads. You can imagine multitasking as something that allows processes to run concurrently, while multithreading allows sub-processes to run concurrently. When multiple threads are running concurrently, this is known as multithreading, which is similar to multitasking. Basically, an operating system with multitasking capabilities allows programs (or processes) to run seemingly at the same time. On the other hand, a single program with multithreading capabilities allows individual sub-processes (or threads) to run seemingly at the same time. One example of multithreading is downloading a video while playing it at the same time. Multithreading is also used extensively in computer-generated animation. Among the widely-used programming languages that allow developers to work on threads in their program source code are Java, Python and .NET."

M1 = preprocess(M1)
M2 = preprocess(M2)

In [6]:
df = pd.read_csv('Store.csv')
df = df.drop(['id'], axis = 1)
df = df.drop([1], axis = 0)
df = df.set_index('Roll Number')

In [7]:
li = []

ct = len(df.index)
for i in range(ct):
    RollNumber = df.index[i]
    A1 = df.loc[df.index[i]][0]
    A2 = df.loc[df.index[i]][1]
    
    A1 = preprocess(A1)
    A2 = preprocess(A2)
    
    vector1 = np.mean([model[word] for word in M1], axis=0)
    vector2 = np.mean([model[word] for word in A1], axis=0)
    
    vector3 = np.mean([model[word] for word in M2], axis=0)
    vector4 = np.mean([model[word] for word in A2], axis=0)
    
    cosine = distance.cosine(vector1, vector2)
    sim1 = round((1-cosine)*100, 2)
    
    cosine = distance.cosine(vector3, vector4)
    sim2 = round((1-cosine)*100, 2)
    
    li.append([RollNumber, sim1, sim2])

/home/aman/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/aman/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [8]:
df = pd.DataFrame(li)
df.fillna(0, inplace = True)
df = df.rename(columns = {0:'RollNumber', 1:'Similarity Score Answer 1', 2:'Similarity Score Answer 2'})
df = df.set_index("RollNumber")
df

,Similarity Score Answer 1,Similarity Score Answer 2
RollNumber,,
testuser,0.00,25.40
1234,0.00,0.00
4301,0.00,10.85
4302,90.72,88.98
4304,73.85,85.64
4305,17.49,0.00
4306,0.00,0.00
